In [ ]:
import os

src_real = "../data/20240607_JMI/real-20240607T081433Z-001/real"
post_path_synth = "../data/20240607_JMI/syn-20240607T081131Z-001/syn"
mask_path = "../data/20240607_JMI/masks/masks/test"
pre_path_real = os.path.join(src_real, "test_A", "test_A")
post_mask_path_real = os.path.join(mask_path, "test_B")

files, cases = [os.path.splitext(i)[0] for i in sorted(set(os.listdir(post_mask_path_real)))], []
for f in files:
    f_splits = f.rsplit("_", 4)
    cases.append(f_splits[1])
s
cases = sorted(set(cases))

Delete files for mask duplicates

In [ ]:
cases_mask_nr = []
for case in cases:
    case_files = [i for i in files if f"_{case}_" in i]
    mask_nr = set()
    for case_file in case_files: 
        mask_nr.add(case_file.rsplit("_", 3)[1])
    cases_mask_nr.append(list(mask_nr))
# cases_mask_nr

for case in cases:
    case_files = [i for i in files if f"_{case}_" in i]
    for case_file in case_files: 
        if case_file.rsplit("_", 3)[1] == "0002":
            os.remove(os.path.join(post_mask_path_real, f"{case_file}.png"))

Rename files for the real phases

In [ ]:
for i in range(3):
    real_phase_path = os.path.join(src_real, f"phase{i+1}", f"phase{i+1}")
    real_files = os.listdir(real_phase_path)
    for file in real_files:
        os.rename(os.path.join(real_phase_path, file), os.path.join(real_phase_path, file.replace("_CONCAT", "").replace("_removed", "")))

Calculate intensities

In [ ]:
from PIL import Image
import numpy as np 
import pandas as pd
from sklearn.metrics import mean_squared_error

width, height = 512, 512
phases, data_intensity, data_mse = ["0000", "0001", "0002", "0003"], [], []
for c in cases: 
    slices = sorted(set([i.rsplit("_", 2)[2].replace("mask", "") for i in  [j for j in files if f"_{c}_" in j]]))
    
    for ph in phases: 
        phase_slices_real_mean, phase_slices_synth_mean, phase_slices_real_std, phase_slices_synth_std, phase_slices_real_norm_mean, phase_slices_synth_norm_mean  = [], [], [], [], [], []
        phase_slices_mse_real_post_synth_post, phase_slices_mse_real_post_real_pre = [], []
        for s in slices:
            mask = Image.open(os.path.join(post_mask_path_real, f"Breast_MRI_{c}_0001_0001_mask{s}.png"))
            if np.array(mask).size and np.count_nonzero(np.array(mask)):
                if ph == "0000":
                    slice_pre_real = Image.open(os.path.join(pre_path_real, f"Breast_MRI_{c}_{ph}_slice{s}.png"))
                    
                    slice_pre_real_tumor = np.array(slice_pre_real)[np.array(mask) != 0]
                    slice_pre_real_non_tumor = np.array(slice_pre_real)[np.array(mask) == 0]
                    
                    phase_slices_real_mean.append(np.mean(slice_pre_real_tumor))
                    phase_slices_real_std.append(np.std(slice_pre_real_tumor))
                    phase_slices_real_norm_mean.append(np.mean(slice_pre_real_tumor)/np.mean(slice_pre_real_non_tumor))
                else:
                    slice_post_real = Image.open(os.path.join(src_real, f"phase{ph[-1]}", f"phase{ph[-1]}", f"Breast_MRI_{c}_{ph}_slice{s}.png"))
                    slice_post_synth = Image.open(os.path.join(post_path_synth, f"phase{ph[-1]}", f"phase{ph[-1]}", f"Breast_MRI_{c}_0000_{ph}_slice{s}.jpg")).convert('L')
                    
                    slice_post_real_tumor = np.array(slice_post_real)[np.array(mask) != 0]
                    slice_post_synth_tumor = np.array(slice_post_synth)[np.array(mask) != 0]
                    slice_post_real_non_tumor = np.array(slice_post_real)[np.array(mask) == 0]
                    slice_post_synth_non_tumor = np.array(slice_post_synth)[np.array(mask) == 0]
                    
                    phase_slices_real_mean.append(np.mean(slice_post_real_tumor))
                    phase_slices_synth_mean.append(np.mean(slice_post_synth_tumor))
                    phase_slices_real_std.append(np.std(slice_post_real_tumor))
                    phase_slices_synth_std.append(np.std(slice_post_synth_tumor))
                    phase_slices_real_norm_mean.append(np.mean(slice_post_real_tumor)/np.mean(slice_post_real_non_tumor))
                    phase_slices_synth_norm_mean.append(np.mean(slice_post_synth_tumor)/np.mean(slice_post_synth_non_tumor))

                    # MSE
                    phase_slices_mse_real_post_synth_post.append(mean_squared_error(slice_post_real_tumor, slice_post_synth_tumor))

                    slice_pre_real = Image.open(os.path.join(pre_path_real, f"Breast_MRI_{c}_0000_slice{s}.png"))
                    slice_pre_real_tumor = np.array(slice_pre_real)[np.nonzero(np.array(mask))]
                    phase_slices_mse_real_post_real_pre.append(mean_squared_error(slice_post_real_tumor, slice_pre_real_tumor))

        if phase_slices_real_mean:   
            phase = "Pre-contrast" if ph == "0000" else int(ph)
            data_intensity.append([c, phase, np.mean(phase_slices_real_mean), np.mean(phase_slices_real_std), np.mean(phase_slices_real_norm_mean), np.std(phase_slices_real_norm_mean), "Real"])
            if ph != "0000":
                data_intensity.append([c, int(ph), np.mean(phase_slices_synth_mean), np.mean(phase_slices_synth_std), np.mean(phase_slices_synth_norm_mean), np.std(phase_slices_synth_norm_mean), "Synthetic"])
                
                data_mse.append([c, int(ph), np.mean(phase_slices_mse_real_post_synth_post), np.std(phase_slices_mse_real_post_synth_post), "Real post-contr. vs. Synthetic post-contr."])
                data_mse.append([c, int(ph), np.mean(phase_slices_mse_real_post_real_pre), np.std(phase_slices_mse_real_post_real_pre), "Real post-contr. vs. Real pre-contr."])

df_intensity = pd.DataFrame(data_intensity, columns=["Breast_MRI", "Phase", "Mean pixel intensity", "Stdev", "Normalized mean pixel intensity", "Stdev norm", "Type"])
df_mse = pd.DataFrame(data_mse, columns=["Breast_MRI", "Phase", "Mean MSE", "Standard deviation", "Type"])

In [ ]:
# df_intensity = df_intensity.rename(columns={"Stdev mean": "Stdev", "Stdev": "Stdev norm"})
# df_intensity

Intensity plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

palette = [".85"] + sns.color_palette(sns.cubehelix_palette(), n_colors=4)
fig, ax = plt.subplots(figsize=(21, 7))

ax = sns.scatterplot(data=df_intensity, x="Breast_MRI", y="Mean pixel intensity", style="Type", hue="Phase", size="Stdev", sizes=(20, 200), palette=palette, legend="brief") 
for case, i in zip(df_intensity.Breast_MRI.unique().tolist(), range(len(df_intensity.Breast_MRI.unique().tolist()))):
    plt.plot([i, i], [df_intensity[(df_intensity.Breast_MRI==case) & (df_intensity.Phase==1) & (df_intensity.Type=="Real")]["Mean pixel intensity"].values[0], df_intensity[(df_intensity.Breast_MRI==case) & (df_intensity.Phase==1) & (df_intensity.Type=="Synthetic")]["Mean pixel intensity"].values[0]], 
             linewidth=1, color=palette[1], zorder=0)
    
handles, lables = ax.get_legend_handles_labels()
for h in handles[0:6]+handles[12:]:
    h.set_markersize(10)
lgnd = plt.legend(handles, lables, bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=0)

sns.set_style("ticks")
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

plt.xlabel('Test case', fontsize=14)
plt.xticks(rotation=90)
plt.ylabel('Mean pixel intensity', fontsize=14)
plt.show()
# plt.savefig('plots/JMI/20240607_JMI_scatter_real_synthetic_MEAN_INTENSITY_all_phases.png', bbox_inches='tight', dpi=300)

MSE plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

palette = sns.color_palette(sns.cubehelix_palette(), n_colors=4)
fig, ax = plt.subplots(figsize=(21, 7))

ax = sns.scatterplot(data=df_mse, x="Breast_MRI", y="Mean MSE", style="Type", hue="Phase", size="Standard deviation", sizes=(20, 200), palette=palette, legend="brief") 
    
handles, lables = ax.get_legend_handles_labels()
for h in handles[0:6]+handles[12:]:
    h.set_markersize(10)
lgnd = plt.legend(handles, lables, bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=0)

sns.set_style("ticks")
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

plt.xlabel('Test case', fontsize=14)
plt.xticks(rotation=90)
plt.ylabel('Mean Squared Error (MSE) in the tumor area', fontsize=14)
plt.show()
# plt.savefig('plots/JMI/20240607_JMI_scatter_MSE_all_phases.png', bbox_inches='tight', dpi=300)

A plot with y-axis = mean intensity value, x-axis = phase in range [1,4] and visualized are for each phase accumulated over cases:  
a) real postcontrast mean intensity value as point (circle size=stdev)  
b) syn postcontrast mean intensity value as point (circle size=stdev)  
c) real pre-contrast mean intensity value (circle size = stdev).  

The circles from (c) will be in a straight line below the other two curves from (a) and (b).

In [ ]:
df_intensity_all = df_intensity.drop(columns=["Breast_MRI", "Normalized mean pixel intensity", "Stdev norm"])

df_a = df_intensity_all[(df_intensity_all.Phase != "Pre-contrast") & (df_intensity_all.Type == "Real")].drop(columns=["Type"])
a = df_a.groupby(by="Phase").mean()
if "Phase" not in a.columns:
    a["Phase"] = a.index.values
    a["Type"] = a.shape[0]*["Real"]

df_b = df_intensity_all[(df_intensity_all.Phase != "Pre-contrast") & (df_intensity_all.Type == "Synthetic")].drop(columns=["Type"])
b = df_b.groupby(by="Phase").mean()
if "Phase" not in b.columns:
    b["Phase"] = b.index.values
    b["Type"] = b.shape[0]*["Synthetic"]

df = pd.concat([a, b], ignore_index=True)

df_pre = df_intensity_all[(df_intensity_all.Phase == "Pre-contrast")].drop(columns=["Type"]).groupby(by="Phase").mean()
df_pre_ = pd.DataFrame(np.repeat(df_pre.values, 3, axis=0))
df_pre_.columns = df_pre.columns
df_pre_["Phase"] = list(range(1, 4))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

palette = [".85"] + sns.color_palette(sns.cubehelix_palette(), n_colors=4)
fig, ax = plt.subplots(figsize=(7, 4))

ax = sns.scatterplot(data=df, x="Phase", y="Mean pixel intensity", hue= "Phase", style="Type", size="Stdev", sizes=(50, 500), legend=False)
ax = sns.scatterplot(data=df_pre_, x="Phase", y="Mean pixel intensity", color = "0.55", size="Stdev", sizes=(50, 500), legend=False)
ax.annotate("Real Pre-contrast", (2.4, 35.5), color="0.55", size=14) # (3.2, 4.12)
ax.annotate("Real Post-contrast", (1.1, 46), color=palette[3], size=14) # (1.5, 6.1)
ax.annotate("Synthetic Post-contrast", (1.6, 41), color=palette[3], size=14) # (2, 4.9)

for i in range(2):
    plt.plot([i+1, i+2], [df[(df.Type=="Real") & (df.Phase==i+1)]["Mean pixel intensity"].values[0], df[(df.Type=="Real") & (df.Phase==i+2)]["Mean pixel intensity"].values[0]], 
             linewidth=4, color=palette[i+1], zorder=0)
    plt.plot([i+1, i+2], [df[(df.Type=="Synthetic") & (df.Phase==i+1)]["Mean pixel intensity"].values[0], df[(df.Type=="Synthetic") & (df.Phase==i+2)]["Mean pixel intensity"].values[0]], 
             linewidth=4, color=palette[i+1], zorder=0)
    plt.plot([i+1, i+2], [df_pre_["Mean pixel intensity"].unique()[0], df_pre_["Mean pixel intensity"].unique()[0]], 
             linewidth=3, color="0.55", zorder=0)
    

ax.set_xticks(list(range(1, 4)))
ax.set_xticklabels(['1','2','3'], fontsize=14)
ax.set_yticks(np.arange(30, 55, 5).tolist())
ax.set_yticklabels(["30", "35", "40","45", ""], fontsize=14)

sns.set_style("ticks")
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

plt.xlabel("Post-Contrast Phase", fontsize=16)
plt.ylabel("Mean pixel intensity", fontsize=16)
plt.show()
# plt.savefig("plots/JMI/20240607_JMI_scatter_real_synthetic_INTENSITY_all_phases_accumulated_test_set.png", bbox_inches='tight', dpi=300)

Boxplot and pointplot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

palette = [".85"] + sns.color_palette(sns.cubehelix_palette(), n_colors=4)
fig, ax = plt.subplots(figsize=(7, 4))

ax = sns.boxplot(data=df_intensity, x="Phase", y="Normalized mean pixel intensity", hue= "Type", palette=palette)
    
plt.setp(ax.get_legend().get_texts(), fontsize='12') # for legend text
# plt.setp(ax.get_legend().get_title(), fontsize='32') # for legend title

ax.set_xticklabels(['Pre-contrast', '1','2','3'], fontsize=14)
ax.set_yticks(np.arange(0, 12, 2).tolist())
ax.set_yticklabels(['','2','4','6', '8', '10'], fontsize=14)

sns.set_style("ticks")
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

plt.xlabel("Post-Contrast Phase", fontsize=16)
plt.ylabel("Normalized mean pixel intensity", fontsize=16)
plt.show()
# plt.savefig("plots/JMI/20240607_JMI_boxplot_real_synthetic_NORMALIZED_INTENSITY_all_phases_accumulated_test_set.png", bbox_inches='tight', dpi=300)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

palette = [".85"] + sns.color_palette(sns.cubehelix_palette(), n_colors=4)
fig, ax = plt.subplots(figsize=(7, 4))

ax = sns.pointplot(data=df_intensity[df_intensity.Phase!="Pre-contrast"], x="Phase", y="Normalized mean pixel intensity", hue= "Type", palette=palette)

ax.set_xticklabels(['1','2','3'], fontsize=14)
ax.set_yticks(np.arange(3, 5.5, .5).tolist())
ax.set_yticklabels(['3.0','3.5','4.0','4.5', '5.0'], fontsize=14)

sns.set_style("ticks")
sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

plt.xlabel("Post-Contrast Phase", fontsize=16)
plt.ylabel("Normalized mean pixel intensity", fontsize=16)
plt.show()
# plt.savefig("plots/JMI/20240607_JMI_pointplot_real_synthetic_NORMALIZED_INTENSITY_all_phases_accumulated_test_set.png", bbox_inches='tight', dpi=300)